# Demo 3: AIS Maritime Data

**Processing ship tracking data from NOAA**

## What is AIS?
- **Automatic Identification System** - GPS tracking for ships
- Ships broadcast position, speed, heading, vessel type
- NOAA collects and provides historical data

## What We're Doing:
- Load AIS data for Port of LA region
- Extract ship counts and vessel types
- Visualize maritime traffic patterns

---

In [ ]:
# Setup - Works both locally and in SageMaker
import sys
import os
from pathlib import Path

# Install dependencies in SageMaker
IS_SAGEMAKER = os.path.exists('/home/ec2-user/SageMaker') or os.environ.get('SM_MODEL_DIR') is not None

if IS_SAGEMAKER:
 print(' Installing dependencies...')
 import subprocess
 subprocess.run(['pip', 'install', 'ultralytics', 'opencv-python-headless', '-q'], check=True)
 print(' Dependencies installed')

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# YOLO import
try:
 from ultralytics import YOLO
 YOLO_AVAILABLE = True
except ImportError:
 YOLO_AVAILABLE = False
 print(' YOLO not available - run: pip install ultralytics')

# Environment detection
if IS_SAGEMAKER:
 PROJECT_ROOT = Path('/home/ec2-user/SageMaker/Real-Time-Economic-Forecasting')
 USE_S3 = True
 print(' Running in AWS SageMaker')
else:
 PROJECT_ROOT = Path.cwd().parent.parent
 USE_S3 = False
 print(' Running locally')

# ===========================================
# S3 BUCKET CONFIGURATION (ACTUAL STRUCTURE)
# ===========================================
S3_RAW = 'economic-forecast-raw'
S3_MODELS = 'economic-forecast-models'
S3_PROCESSED = 'economic-forecast-processed'

# S3 Paths (matching actual bucket structure)
S3_PATHS = {
 'satellite': f's3://{S3_RAW}/satellite/google_earth',
 'port_la_images': f's3://{S3_RAW}/satellite/google_earth/Port_of_LA',
 'mall_images': f's3://{S3_RAW}/satellite/google_earth/Mall_of_america',
 'models': f's3://{S3_MODELS}/yolo',
 'port_model': f's3://{S3_MODELS}/yolo/ports/best.pt',
 'retail_model': f's3://{S3_MODELS}/yolo/retail/best.pt',
 'city_model': f's3://{S3_MODELS}/yolo/city/best.pt',
 'ais': f's3://{S3_PROCESSED}/ais',
 'ais_la': f's3://{S3_PROCESSED}/ais/Port_of_LA_ais_features.csv',
 'detections': f's3://{S3_PROCESSED}/detections',
 'news': f's3://{S3_RAW}/news/sentiment/data',
}

# Local paths
LOCAL_PATHS = {
 'satellite': PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth',
 'port_la_images': PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Port_of_LA',
 'mall_images': PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Mall_of_america',
 'models': PROJECT_ROOT / 'data' / 'models' / 'satellite',
 'port_model': PROJECT_ROOT / 'data' / 'models' / 'satellite' / 'ports_dota_yolo11_20251127_013205' / 'weights' / 'best.pt',
 'retail_model': PROJECT_ROOT / 'data' / 'models' / 'satellite' / 'retail_yolo11_20251126_150811' / 'weights' / 'best.pt',
 'ais': PROJECT_ROOT / 'data' / 'processed' / 'ais',
 'ais_la': PROJECT_ROOT / 'data' / 'processed' / 'ais' / 'Port_of_LA_ais_features.csv',
 'detections': PROJECT_ROOT / 'results' / 'annotations',
}

def get_path(key):
 '''Get path - S3 or local based on environment.'''
 if USE_S3:
  return S3_PATHS.get(key, S3_PATHS.get('satellite'))
 else:
  return LOCAL_PATHS.get(key, LOCAL_PATHS.get('satellite'))

def download_model(model_type='port'):
 '''Download model from S3 to local temp for inference.'''
 if not USE_S3:
  # Return local path
  if model_type == 'port':
   return LOCAL_PATHS['port_model']
  elif model_type == 'retail':
   return LOCAL_PATHS['retail_model']
  return None
 
 import boto3
 import tempfile
 
 s3 = boto3.client('s3')
 
 model_keys = {
  'port': 'yolo/ports/best.pt',
  'retail': 'yolo/retail/best.pt',
  'city': 'yolo/city/best.pt',
 }
 
 key = model_keys.get(model_type)
 if not key:
  print(f' Unknown model type: {model_type}')
  return None
 
 local_path = Path(tempfile.gettempdir()) / f'{model_type}_best.pt'
 
 if not local_path.exists():
  print(f' Downloading {model_type} model from S3...')
  s3.download_file(S3_MODELS, key, str(local_path))
  print(f' Model saved to {local_path}')
 else:
  print(f' Using cached model: {local_path}')
 
 return local_path

def list_s3_images(prefix):
 '''List images in S3 bucket.'''
 import boto3
 s3 = boto3.client('s3')
 
 # Parse bucket and prefix from s3:// path
 if prefix.startswith('s3://'):
  parts = prefix.replace('s3://', '').split('/', 1)
  bucket = parts[0]
  prefix = parts[1] if len(parts) > 1 else ''
 else:
  bucket = S3_RAW
 
 response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
 
 images = []
 for obj in response.get('Contents', []):
  key = obj['Key']
  if key.endswith(('.jpg', '.jpeg', '.png', '.tif')):
   images.append(f's3://{bucket}/{key}')
 
 return images

def download_image(s3_path, local_dir='/tmp'):
 '''Download single image from S3.'''
 import boto3
 s3 = boto3.client('s3')
 
 parts = s3_path.replace('s3://', '').split('/', 1)
 bucket = parts[0]
 key = parts[1]
 
 filename = key.split('/')[-1]
 local_path = Path(local_dir) / filename
 
 s3.download_file(bucket, key, str(local_path))
 return local_path

print(f' Setup complete | S3: {USE_S3} | YOLO: {YOLO_AVAILABLE}')
print(f' Project: {PROJECT_ROOT}')


---
## Understanding AIS Data

```
Ship broadcasts every few seconds:
┌─────────────────────────────────────────────────┐
│ MMSI: 123456789 (unique ship ID)    │
│ LAT: 33.7234, LON: -118.2654     │
│ SOG: 12.5 knots (speed over ground)    │
│ COG: 275° (course over ground)     │
│ VesselType: 70 (cargo ship)      │
│ VesselName: "EVER GIVEN"      │
└─────────────────────────────────────────────────┘
```

In [ ]:
# AIS Vessel Type Codes
print("📋 AIS VESSEL TYPE CODES")
print("="*50)

vessel_types = {
 '70-79': 'Cargo vessels',
 '80-89': 'Tankers',
 '60-69': 'Passenger vessels',
 '30': 'Fishing vessels',
 '31-32': 'Tug boats',
 '36-37': 'Sailing vessels',
}

for code, desc in vessel_types.items():
 print(f" {code}: {desc}")

print("\n Cargo (70-79) and Tankers (80-89) are key economic indicators!")

---
## Load AIS Data

In [ ]:
# Check available AIS data
ais_processed_dir = PROJECT_ROOT / 'data' / 'processed' / 'ais'
ais_raw_dir = PROJECT_ROOT / 'data' / 'raw' / 'ais' / 'noaa_daily'

print("AVAILABLE AIS DATA")
print("="*50)

# Check processed data first (ready to use)
print("\nProcessed AIS Data:")
if ais_processed_dir.exists():
    for f in sorted(ais_processed_dir.glob('*.csv')):
        size = f.stat().st_size / 1024
        print(f"  {f.name} ({size:.1f} KB)")
else:
    print("  No processed data found")

# Check raw data
print("\nRaw AIS Data (zip files):")
if ais_raw_dir.exists():
    for year_dir in sorted(ais_raw_dir.iterdir()):
        if year_dir.is_dir():
            files = list(year_dir.glob('*.zip'))
            if files:
                total_size = sum(f.stat().st_size for f in files) / (1024*1024)
                print(f"  {year_dir.name}: {len(files)} files ({total_size:.1f} MB)")
else:
    print("  No raw data found")

In [ ]:
# Port of LA bounding box
PORT_LA_BOUNDS = {
 'min_lat': 33.65,
 'max_lat': 33.85,
 'min_lon': -118.35,
 'max_lon': -118.15
}

print("📍 PORT OF LA BOUNDING BOX")
print("="*50)
print(f" Latitude: {PORT_LA_BOUNDS['min_lat']}° to {PORT_LA_BOUNDS['max_lat']}°")
print(f" Longitude: {PORT_LA_BOUNDS['min_lon']}° to {PORT_LA_BOUNDS['max_lon']}°")

# Visualize on a simple map
fig, ax = plt.subplots(figsize=(10, 8))

# Draw bounding box
from matplotlib.patches import Rectangle
rect = Rectangle(
 (PORT_LA_BOUNDS['min_lon'], PORT_LA_BOUNDS['min_lat']),
 PORT_LA_BOUNDS['max_lon'] - PORT_LA_BOUNDS['min_lon'],
 PORT_LA_BOUNDS['max_lat'] - PORT_LA_BOUNDS['min_lat'],
 fill=True, facecolor='lightblue', edgecolor='blue', linewidth=2, alpha=0.5
)
ax.add_patch(rect)

# Add port marker
ax.plot(-118.25, 33.75, 'r*', markersize=20, label='Port of LA')

ax.set_xlim(-118.5, -118.0)
ax.set_ylim(33.5, 34.0)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_title('🗺️ Port of LA - AIS Capture Area', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

---
## Process AIS Sample Data

In [ ]:
# Load ACTUAL processed AIS data
import zipfile

def load_ais_data():
    """Load AIS data - processed CSV or from zip file."""
    
    # First try processed data (preferred)
    processed_files = {
        'daily': PROJECT_ROOT / 'data' / 'processed' / 'ais' / 'Port_of_LA_ais_daily.csv',
        'features': PROJECT_ROOT / 'data' / 'processed' / 'ais' / 'Port_of_LA_ais_features.csv',
    }
    
    if processed_files['daily'].exists():
        print(f"Loading processed data: {processed_files['daily'].name}")
        df = pd.read_csv(processed_files['daily'])
        print(f"  Loaded {len(df):,} daily records")
        return df, 'daily'
    
    if processed_files['features'].exists():
        print(f"Loading features data: {processed_files['features'].name}")
        df = pd.read_csv(processed_files['features'])
        print(f"  Loaded {len(df):,} feature records")
        return df, 'features'
    
    # Try raw zip files
    ais_files = list((PROJECT_ROOT / 'data' / 'raw' / 'ais' / 'noaa_daily').rglob('*.zip'))
    
    if ais_files:
        zip_path = ais_files[0]
        print(f"Loading raw data: {zip_path.name}")
        
        with zipfile.ZipFile(zip_path, 'r') as z:
            csv_name = [f for f in z.namelist() if f.endswith('.csv')][0]
            with z.open(csv_name) as f:
                df = pd.read_csv(f, nrows=100000)
        
        # Filter to Port of LA
        df_filtered = df[
            (df['LAT'] >= PORT_LA_BOUNDS['min_lat']) &
            (df['LAT'] <= PORT_LA_BOUNDS['max_lat']) &
            (df['LON'] >= PORT_LA_BOUNDS['min_lon']) &
            (df['LON'] <= PORT_LA_BOUNDS['max_lon'])
        ]
        
        if len(df_filtered) > 0:
            return df_filtered, 'raw'
    
    # Create sample if nothing found
    print("Creating sample AIS data for demo...")
    np.random.seed(42)
    n_records = 365
    
    sample_df = pd.DataFrame({
        'date': pd.date_range('2024-01-01', periods=n_records, freq='D'),
        'unique_ships': np.random.randint(150, 300, n_records),
        'cargo_ships': np.random.randint(80, 150, n_records),
        'tanker_ships': np.random.randint(30, 60, n_records),
        'avg_speed': np.random.uniform(8, 12, n_records),
    })
    
    return sample_df, 'sample'

ais_df, data_type = load_ais_data()
print(f"\nData type: {data_type}")
print(f"Columns: {list(ais_df.columns)}")

In [ ]:
# Show AIS data
print("AIS DATA")
print("="*60)
display(ais_df)

---
## Analyze Ship Traffic

In [ ]:
# Analyze ship traffic from ACTUAL data
print("SHIP TRAFFIC ANALYSIS")
print("="*50)

if 'unique_ships' in ais_df.columns:
    # Using processed daily data
    print("\nYearly Ship Counts at Port of LA:")
    print("-"*40)
    
    for _, row in ais_df.iterrows():
        date = row['date'] if isinstance(row['date'], str) else str(row['date'])[:10]
        year = date[:4]
        print(f"  {date}: {row['unique_ships']} total ships")
        print(f"           Cargo: {row['cargo_ships']}, Tankers: {row['tanker_ships']}, Passenger: {row['passenger_ships']}")
    
    # Calculate averages
    avg_ships = ais_df['unique_ships'].mean()
    avg_cargo = ais_df['cargo_ships'].mean()
    avg_tanker = ais_df['tanker_ships'].mean()
    
    print(f"\nAverages (2018-2024):")
    print(f"  Total ships/day: {avg_ships:.0f}")
    print(f"  Cargo ships/day: {avg_cargo:.0f}")
    print(f"  Tanker ships/day: {avg_tanker:.0f}")
else:
    # Raw data with vessel categories
    ship_counts = ais_df.groupby('vessel_category')['MMSI'].nunique().sort_values(ascending=False)
    for cat, count in ship_counts.items():
        print(f"  {cat}: {count} ships")

In [ ]:
# Visualize ACTUAL vessel distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if 'unique_ships' in ais_df.columns:
    # Using processed daily data
    years = [str(d)[:4] for d in ais_df['date']]
    
    # Bar chart of total ships by year
    ax1 = axes[0]
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(ais_df)))
    bars = ax1.bar(years, ais_df['unique_ships'], color=colors, edgecolor='black')
    ax1.set_xlabel('Year', fontsize=12)
    ax1.set_ylabel('Unique Ships', fontsize=12)
    ax1.set_title('Total Ships at Port of LA', fontsize=14, fontweight='bold')
    for bar, val in zip(bars, ais_df['unique_ships']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3, str(val), 
                ha='center', fontweight='bold')
    
    # Stacked bar of vessel types
    ax2 = axes[1]
    x = np.arange(len(years))
    width = 0.6
    
    ax2.bar(x, ais_df['cargo_ships'], width, label='Cargo', color='#2ecc71')
    ax2.bar(x, ais_df['tanker_ships'], width, bottom=ais_df['cargo_ships'], label='Tanker', color='#3498db')
    ax2.bar(x, ais_df['passenger_ships'], width, 
            bottom=ais_df['cargo_ships'] + ais_df['tanker_ships'], label='Passenger', color='#e74c3c')
    
    ax2.set_xlabel('Year', fontsize=12)
    ax2.set_ylabel('Number of Ships', fontsize=12)
    ax2.set_title('Ships by Type', fontsize=14, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(years)
    ax2.legend()

plt.tight_layout()
plt.show()

# COVID impact analysis
if 'unique_ships' in ais_df.columns:
    print("\nCOVID-19 Impact Analysis:")
    print("-"*40)
    pre_covid = ais_df[ais_df['date'].astype(str) < '2020']['unique_ships'].mean()
    covid_2020 = ais_df[ais_df['date'].astype(str).str.startswith('2020')]['unique_ships'].values
    post_covid = ais_df[ais_df['date'].astype(str) > '2021']['unique_ships'].mean()
    
    if len(covid_2020) > 0:
        print(f"  Pre-COVID (2018-2019): {pre_covid:.0f} ships/day")
        print(f"  During COVID (2020):   {covid_2020[0]} ships/day")
        print(f"  Post-COVID (2022-2024): {post_covid:.0f} ships/day")
        print(f"\n  COVID drop: {((pre_covid - covid_2020[0])/pre_covid)*100:.1f}%")
        print(f"  Recovery: +{((post_covid - covid_2020[0])/covid_2020[0])*100:.1f}%")

---
## Ship Positions Map

In [ ]:
# Ship traffic trends over time
fig, ax = plt.subplots(figsize=(12, 6))

if 'unique_ships' in ais_df.columns:
    years = [str(d)[:4] for d in ais_df['date']]
    
    ax.plot(years, ais_df['unique_ships'], 'b-o', linewidth=2, markersize=10, label='Total Ships')
    ax.plot(years, ais_df['cargo_ships'], 'g--s', linewidth=2, markersize=8, label='Cargo Ships')
    ax.plot(years, ais_df['tanker_ships'], 'r--^', linewidth=2, markersize=8, label='Tankers')
    
    # Highlight COVID year
    covid_idx = [i for i, y in enumerate(years) if y == '2020']
    if covid_idx:
        ax.axvspan(covid_idx[0]-0.3, covid_idx[0]+0.3, alpha=0.2, color='red', label='COVID-19')
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Number of Ships', fontsize=12)
    ax.set_title('Port of LA - Ship Traffic Trends (2018-2024)', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Economic Metrics from AIS

In [ ]:
# Calculate economic indicators from ACTUAL data
print("ECONOMIC INDICATORS FROM AIS DATA")
print("="*60)

if 'unique_ships' in ais_df.columns:
    # Calculate metrics
    total_avg = ais_df['unique_ships'].mean()
    cargo_avg = ais_df['cargo_ships'].mean()
    tanker_avg = ais_df['tanker_ships'].mean()
    
    # Trade activity index
    trade_ships = ais_df['cargo_ships'] + ais_df['tanker_ships']
    trade_ratio = (trade_ships / ais_df['unique_ships'] * 100).mean()
    
    # Year-over-year growth
    if len(ais_df) > 1:
        yoy_growth = ((ais_df['unique_ships'].iloc[-1] - ais_df['unique_ships'].iloc[0]) / 
                      ais_df['unique_ships'].iloc[0] * 100)
    else:
        yoy_growth = 0
    
    print(f"\nKey Metrics (2018-2024):")
    print(f"  Average ships/day:     {total_avg:.0f}")
    print(f"  Average cargo ships:   {cargo_avg:.0f}")
    print(f"  Average tankers:       {tanker_avg:.0f}")
    print(f"  Trade Activity Ratio:  {trade_ratio:.1f}%")
    print(f"  Growth (2018→2024):    {yoy_growth:+.1f}%")
    
    print(f"\nLatest Data ({ais_df['date'].iloc[-1]}):")
    latest = ais_df.iloc[-1]
    print(f"  Total ships:    {latest['unique_ships']}")
    print(f"  Cargo ships:    {latest['cargo_ships']}")
    print(f"  Tanker ships:   {latest['tanker_ships']}")
    print(f"  Passenger:      {latest['passenger_ships']}")
    
    print(f"\nINSIGHT:")
    print(f"  Port traffic recovered {yoy_growth:.0f}% since 2018")
    print(f"  Cargo ships represent {cargo_avg/total_avg*100:.0f}% of trade activity")

---
## Summary

### Data Source:
- **NOAA AIS Data**: Ship tracking from 2018-2024
- **Port of LA**: Major US trade gateway
- **7 sample days**: One per year for trend analysis

### Key Findings:
| Year | Total Ships | Cargo | Tankers |
|------|-------------|-------|---------|
| 2018 | 234 | 37 | 12 |
| 2019 | 239 | 27 | 15 |
| 2020 | 218 | 21 | 7 | (COVID drop)
| 2021 | 275 | 57 | 18 |
| 2022 | 257 | 59 | 15 |
| 2023 | 286 | 44 | 15 |
| 2024 | 287 | 42 | 15 |

### Economic Insights:
1. **COVID-19 Impact**: 7% drop in 2020
2. **Recovery**: 32% increase from 2020 to 2024
3. **Trade Activity**: Cargo + Tankers = key economic indicator

### Next Step:
**Demo 4**: Fuse satellite detections with AIS data

In [ ]:
print("="*60)
print(" Demo 3 Complete: AIS Data Processing")
print("="*60)
print("\n Next: Demo_4_Data_Fusion.ipynb")